# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

In [1]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

In [2]:
import collections

import helper
import numpy as np
import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, Flatten, LSTM,RepeatVector
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

Using TensorFlow backend.


### Verify access to the GPU
The following test applies only if you expect to be using a GPU, e.g., while running in a Udacity Workspace or using an AWS instance with GPU support. Run the next cell, and verify that the device_type is "GPU".
- If the device is not GPU & you are running from a Udacity Workspace, then save your workspace with the icon at the top, then click "enable" at the bottom of the workspace.
- If the device is not GPU & you are running from an AWS instance, then refer to the cloud computing instructions in the classroom to verify your setup steps.

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17447907786137852231
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 357433344
locality {
  bus_id: 1
}
incarnation: 12936644726804344643
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0"
]


## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [4]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


In [5]:
english_sentences[0]

'new jersey is sometimes quiet during autumn , and it is snowy in april .'

In [6]:
print(len(english_sentences))
print(len(french_sentences))

137861
137861


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [7]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [8]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [9]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    return tokenizer.texts_to_sequences(x), tokenizer

tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


In [10]:
len(text_tokenized[0])

9

### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

pad_sequences converts list of lists into numpy 2D array.

In [11]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    if not length: 
        length = 0
    for i in range(len(x)):
        length = max(length, len(x[i]))
    padded = pad_sequences(x,maxlen=length, padding="post")
    return padded

tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [12]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print('English sentenses shape:', preproc_english_sentences.shape)
print('French sentences shape:', preproc_french_sentences.shape)
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
English sentenses shape: (137861, 15)
French sentences shape: (137861, 21, 1)
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [13]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [14]:
# from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    learning_rate = 0.01
    # TODO: Build the layers
    inputLayer = Input(shape=input_shape[1:])
    print('Input Layer Shape:',inputLayer.shape)
    GRULayer = GRU(units=64,return_sequences=True)(inputLayer)
    print('GRULayer shape:',GRULayer.shape)
    outputLayer = TimeDistributed(Dense(units=french_vocab_size))(GRULayer)
    print('Output Layer shape:',outputLayer.shape)
    model = Model(inputLayer, Activation('softmax')(outputLayer))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
# tests.test_simple_model(simple_model)?
# 
# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))
print('Input Shape:', tmp_x.shape)
print('Output sequence length:', max_french_sequence_length)
print('English vocab size:', english_vocab_size)
print('French vocab size:', french_vocab_size)
# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
# simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=20, validation_split=0.2)

# Print prediction(s)
# print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Input Shape: (137861, 21, 1)
Output sequence length: 21
English vocab size: 199
French vocab size: 344
Input Layer Shape: (?, 21, 1)
GRULayer shape: (?, ?, 64)
Output Layer shape: (?, 21, 344)


In [15]:
tmp_x.shape

(137861, 21, 1)

Source sentence: 'new jersey is sometimes quiet during autumn , and it is snowy in april .'
    - Translation with GRU (10 Epochs): new jersey est parfois calme en mois de il est est en en 
    - Translation with GRU (10 Epochs): new jersey is sometimes calm in the month of it is in in

In [16]:
# ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])
engDict = {id: word for word, id in english_tokenizer.word_index.items()}
for id in tmp_x[:1][0]:
    index = id[0]
    if index != 0:
        print(engDict[id[0]])

new
jersey
is
sometimes
quiet
during
autumn
and
it
is
snowy
in
april


### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

#### Question: What the input&output are shapes for model.fit method
- Output and input should be consistent
- Since the output is shaped 3 dimensions for calculating sparse categorical crossentropy, input should be similarly sized too.
- Hence reshaping the input 

#### Data:
- Input: 137861 english sentences of length 15 words each
- Output: 137861 french sentences of length 21 words each

#### What is embedding layer input:
- Since embedding layer captures sematic information, the input should be whole sentences. That way relationship between words is also captured.
- Each word should be integer encoded (Keras embedding) 
- Also the output dimensions, how many dimensions each vector should have after embedding.
- input_dim = input vocab length
- output_dim = embedding vector length
- input_len = length of each sentence?

#### What should be embedding layer input shape:
 - [total_num_of_sentences, length_of_each_sentence]

#### What is the embedding layer output
 - 2D vector containing [word_id, embedding_vector]
 - Output contains embedding to each word in the input vocab

In [17]:
print(preproc_english_sentences.shape)
print(preproc_french_sentences.shape)

(137861, 15)
(137861, 21, 1)


In [18]:
inputLayer = Input(shape=[21,])
print(inputLayer.shape)
embedLayer = Embedding(input_dim=english_vocab_size, output_dim=64, input_length=21)(inputLayer)
print(embedLayer.shape)
GRULay = GRU(units=64, return_sequences=True)(embedLayer)
print(GRULay.shape)

(?, 21)
(?, 21, 64)
(?, ?, 64)


- Why to convert input and output sequences to same length?

In [19]:
# from keras.models import Sequential
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    inputLayer = Input(shape=input_shape[1:])
    embeddingLayer = Embedding(input_dim=english_vocab_size,output_dim=64, input_shape=inputLayer.shape)(inputLayer)
    print('Embedding layer shape:',embeddingLayer.shape)
    GRULayer = GRU(units=128, return_sequences=True)(embeddingLayer)
    outputLayer = TimeDistributed(Dense(units=french_vocab_size))(GRULayer)
    model = Model(inputLayer, Activation('softmax')(outputLayer))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

tests.test_embed_model(embed_model)


# TODO: Reshape the input
input_x = pad(preproc_english_sentences, max_french_sequence_length)
print('Input shape prior to reshaping:', input_x.shape)
input_x = input_x.reshape((-1, preproc_french_sentences.shape[-2]))

# TODO: Train the neural network
embeddedModel = embed_model(input_x.shape, max_french_sequence_length, english_vocab_size, french_vocab_size)
# from tensorflow import keras
# keras.utils.plot_model(embeddedModel, "my_first_model.png")
embeddedModel.fit(input_x, preproc_french_sentences, batch_size=1024, epochs=20, validation_split=0.2)

# TODO: Print prediction(s)
print(logits_to_text(embeddedModel.predict(input_x[:1])[0], french_tokenizer))

Embedding layer shape: (?, 21, 64)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 21)                0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 21, 64)            12736     
_________________________________________________________________
gru_3 (GRU)                  (None, 21, 128)           74112     
_________________________________________________________________
time_distributed_2 (TimeDist (None, 21, 344)           44376     
_________________________________________________________________
activation_2 (Activation)    (None, 21, 344)           0         
Total params: 131,224
Trainable params: 131,224
Non-trainable params: 0
_________________________________________________________________
Input shape prior to reshaping: (137861, 21)
Embedding layer shape: (?, 21, 64)
_____________________

In [20]:
# TODO: Print prediction(s)
print(logits_to_text(embeddedModel.predict(input_x[3:4])[0], french_tokenizer))

les états unis est parfois doux en juin et il est froid en septembre <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [21]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    inputLayer = Input(shape=input_shape[1:])
    bdLayer = Bidirectional(LSTM(64, return_sequences=True))(inputLayer)
    outputLayer = TimeDistributed(Dense(french_vocab_size,activation='softmax'))(bdLayer)
    bdmodel = Model(inputs=inputLayer, outputs=outputLayer)
    bdmodel.compile(loss=sparse_categorical_crossentropy, optimizer=Adam(0.01), metrics=['accuracy'])
    return bdmodel
tests.test_bd_model(bd_model)

#Preprocess the input into [samples, timesteps, features_per_each_timestep]
bdinput = pad(preproc_english_sentences, max_french_sequence_length)
# bdinput = preproc_english_sentences
bdinput = bdinput.reshape(-1, bdinput.shape[1],1)

#Build the model
bdmodel = bd_model(bdinput.shape, max_french_sequence_length, english_vocab_size,french_vocab_size)

#Train the model
bdmodel.fit(bdinput, preproc_french_sentences, epochs=20, batch_size=1024, validation_split=0.2)

#Make a prediction after training
print(logits_to_text(bdmodel.predict(tmp_x[3:4])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/20
110288/110288 [==============================] - 13s 114us/step - loss: 1.9134 - acc: 0.5628 - val_loss: nan - val_acc: 0.6306
Epoch 2/20
110288/110288 [==============================] - 12s 108us/step - loss: 1.2398 - acc: 0.6508 - val_loss: nan - val_acc: 0.6652
Epoch 3/20
110288/110288 [==============================] - 12s 109us/step - loss: 1.1150 - acc: 0.6730 - val_loss: nan - val_acc: 0.6745
Epoch 4/20
110288/110288 [==============================] - 12s 110us/step - loss: 1.0439 - acc: 0.6862 - val_loss: nan - val_acc: 0.6897
Epoch 5/20
110288/110288 [==============================] - 12s 109us/step - loss: 0.9922 - acc: 0.6973 - val_loss: nan - val_acc: 0.7080
Epoch 6/20
110288/110288 [==============================] - 12s 109us/step - loss: 0.9514 - acc: 0.7061 - val_loss: nan - val_acc: 0.7128
Epoch 7/20
110288/110288 [==============================] - 12s 109us/step - loss: 0.9212 - acc: 0.7141 - val_loss: nan -

### Model 4: Encoder-Decoder (OPTIONAL)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [22]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """    
    #Encoder
    encoder_input = Input(shape=input_shape[1:])
    print('Encoder input shape:', encoder_input.shape)
    encoder_gru = GRU(256)(encoder_input)
    print('Encoder GRU shape:', encoder_gru.shape)
    decoder_input = RepeatVector(output_sequence_length)(encoder_gru)
    print('Decoder input shape:', decoder_input.shape)
    decoder_gru = GRU(256,return_sequences=True)(decoder_input)
    print('Decoder GRU shape:', decoder_gru.shape)
    decoder_output = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(decoder_gru)
    edmodel = Model(inputs=encoder_input, outputs=decoder_output)

    # Build the encoder-decoder RNN model
#     input_layer= Input(shape=input_shape[1:])
#     encoder = GRU(256, recurrent_dropout=0.2)(input_layer)
#     repeat_vector = RepeatVector(output_sequence_length)(encoder)
#     decoder = GRU(256, return_sequences=True, recurrent_dropout=0.2)(repeat_vector)
#     dense_layer = Dense(french_vocab_size, activation='relu')(decoder)
#     output_layer = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(decoder)
#     edmodel = Model(inputs=input_layer, outputs=output_layer)
    
    # Compile the model
    learning_rate = 0.01
    edmodel.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(lr=learning_rate),
                  metrics=['accuracy'])
    edmodel.summary()
    return edmodel
# tests.test_encdec_model(encdec_model)


# OPTIONAL: Train and Print prediction(s)
edinput = pad(preproc_english_sentences, max_french_sequence_length)
edinput = edinput.reshape((-1, edinput.shape[-1],1))
print('Input shape:', edinput.shape)
print('Output shape:', preproc_french_sentences.shape)
edmodel = encdec_model(edinput.shape, max_french_sequence_length, english_vocab_size, french_vocab_size)
edmodel.fit(edinput,preproc_french_sentences, batch_size=1024, epochs=20, validation_split=0.2)
#Make a prediction after training
print(logits_to_text(edmodel.predict(bdinput[3:4])[0], french_tokenizer))

Input shape: (137861, 21, 1)
Output shape: (137861, 21, 1)
Encoder input shape: (?, 21, 1)
Encoder GRU shape: (?, 256)
Decoder input shape: (?, 21, 256)
Decoder GRU shape: (?, ?, 256)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 21, 1)             0         
_________________________________________________________________
gru_5 (GRU)                  (None, 256)               198144    
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 21, 256)           0         
_________________________________________________________________
gru_6 (GRU)                  (None, 21, 256)           393984    
_________________________________________________________________
time_distributed_6 (TimeDist (None, 21, 344)           88408     
Total params: 680,536
Trainable params: 680,536
Non-trainable params: 0
__________________

### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [23]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    inputLayer = Input(shape=input_shape[1:])
#     embeddingLayer = Embedding(input_dim=english_vocab_size, output_dim=64)(inputLayer)
    embeddingLayer = Embedding(english_vocab_size*2, english_vocab_size)(inputLayer)
    bdLayerIn = Bidirectional(GRU(256))(embeddingLayer)
    repeatlayer = RepeatVector(output_sequence_length)(bdLayerIn)
    denselayer = Dense(256,activation='relu')(repeatlayer)
    bdLayerOut = Bidirectional(GRU(256,return_sequences=True))(denselayer)    
    outputLayer = TimeDistributed(Dense(french_vocab_size,activation='relu'))(bdLayerOut)
    bdmodel = Model(inputs=inputLayer, outputs=outputLayer)
    bdmodel.compile(loss=sparse_categorical_crossentropy, optimizer=Adam(0.005), metrics=['accuracy'])
    return bdmodel    
tests.test_model_final(model_final)


print('Final Model Loaded')

Final Model Loaded


## Prediction (IMPLEMENTATION)

In [24]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    x = pad(x,y.shape[1])

    # Define final_model
    final_model = model_final(x.shape, 
                              y.shape[1],
                              len(x_tk.word_index) + 1,
                              len(y_tk.word_index) + 1)
    # Fit the final model
    final_model.fit(x, y, batch_size=1024, epochs=10, validation_split=0.2)
    
    model = final_model

    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 42s 384us/step - loss: 7.3445 - acc: 0.0249 - val_loss: 6.9764 - val_acc: 0.0168
Epoch 2/10
110288/110288 [==============================] - 41s 373us/step - loss: 6.4024 - acc: 0.2564 - val_loss: 5.8435 - val_acc: 0.4093
Epoch 3/10
110288/110288 [==============================] - 41s 373us/step - loss: 5.8435 - acc: 0.4082 - val_loss: 5.8435 - val_acc: 0.4093
Epoch 4/10
110288/110288 [==============================] - 41s 371us/step - loss: 5.8435 - acc: 0.4082 - val_loss: 5.8435 - val_acc: 0.4093
Epoch 5/10
110288/110288 [==============================] - 41s 372us/step - loss: 5.8435 - acc: 0.4082 - val_loss: 5.8435 - val_acc: 0.4093
Epoch 6/10
110288/110288 [==============================] - 41s 374us/step - loss: 5.8435 - acc: 0.4082 - val_loss: 5.8435 - val_acc: 0.4093
Epoch 7/10
110288/110288 [==============================] - 41s 372us/step - loss: 5.8435 - acc: 0.4082

## Submission
When you're ready to submit, complete the following steps:
1. Review the [rubric](https://review.udacity.com/#!/rubrics/1004/view) to ensure your submission meets all requirements to pass
2. Generate an HTML version of this notebook

  - Run the next cell to attempt automatic generation (this is the recommended method in Workspaces)
  - Navigate to **FILE -> Download as -> HTML (.html)**
  - Manually generate a copy using `nbconvert` from your shell terminal
```
$ pip install nbconvert
$ python -m nbconvert machine_translation.ipynb
```
  
3. Submit the project

  - If you are in a Workspace, simply click the "Submit Project" button (bottom towards the right)
  
  - Otherwise, add the following files into a zip archive and submit them 
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.

### Generate the html

**Save your notebook before running the next cell to generate the HTML output.** Then submit your project.

In [25]:
# Save before you run this cell!
!!jupyter nbconvert *.ipynb

['[NbConvertApp] Converting notebook machine_translation-Copy1.ipynb to html',
 '[NbConvertApp] Writing 331287 bytes to machine_translation-Copy1.html',
 '[NbConvertApp] Converting notebook machine_translation.ipynb to html',
 '[NbConvertApp] Writing 389412 bytes to machine_translation.html',
 '[NbConvertApp] Converting notebook machine_translation-zh.ipynb to html',
 '[NbConvertApp] Writing 328614 bytes to machine_translation-zh.html']

## Optional Enhancements

This project focuses on learning various network architectures for machine translation, but we don't evaluate the models according to best practices by splitting the data into separate test & training sets -- so the model accuracy is overstated. Use the [`sklearn.model_selection.train_test_split()`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to create separate training & test datasets, then retrain each of the models using only the training set and evaluate the prediction accuracy using the hold out test set. Does the "best" model change?